### GNN embedding

In [ ]:
#!pip install python-chess


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 39.1 MB/s eta 0:00:00m eta 0:00:010:01:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147780 sha256=72c3cd4c7dfb6457f076021e77d5a3c34d804cf5258324ae2fba1ae45e427176
  Stored in directory: /Users/franciscomeza/Library/Caches/pip/wheels/6c/68/b2/d8242fcd2fd057e87e1985fdd6e71c04b540fc5fc93ad87545
Successfully built chess

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
#!pip install torch-geometric torch torchvision torch-scatter torch-sparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.8 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp38-cp38-macosx_13_0_arm64.whl size=280488 sha256=8b22eff38263d8cbba3436df8e9eea7e6a57bd3e81d294e6969838f89662e2ab
  Stored in directory: /Users/franciscomeza/Library/Caches/pip/wheels/54/0c/3d/5e4aea36abfd59a43f8bb9859f545baa46c2a880a99a428db6
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp38-cp38-macosx_13_0_arm64.whl size=458859 sha256=2c5be40cd030415ea4e19f0ba3987b5aab34e25e282ee517d1e2b8029a344d89
  Stored in directory: /Users/franciscomeza/Library/Caches/pip/wheels/73/9f/4e/6671faed9dea2e8152688ebb3be08ce52e7c5f3db9a88021d8
Successfully built torch-scatter torch-sp

In [3]:
import os

In [5]:
os.chdir('..')

In [7]:
import os
import zipfile
import torch
import chess.pgn
import numpy as np
from tqdm import tqdm
from torch_geometric.data import Data

# =====================
# CONFIGURACIÓN
# =====================
ZIP_FOLDER = "pgns"              # carpeta con zips
EXTRACT_FOLDER = "temp_pgns"     # carpeta temporal
os.makedirs(EXTRACT_FOLDER, exist_ok=True)

# =====================
# FUNCIÓN DE FEATURES
# =====================

def board_to_feature(board):
    """Convierte el estado del tablero a un vector de 773 features"""
    piece_map = board.piece_map()
    planes = np.zeros((12, 64), dtype=np.float32)
    piece_to_index = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }
    for square, piece in piece_map.items():
        planes[piece_to_index[piece.symbol()]][square] = 1.0
    flat_pieces = planes.reshape(-1)  # 768

    extras = [
        float(board.turn),
        float(board.has_kingside_castling_rights(chess.WHITE)),
        float(board.has_queenside_castling_rights(chess.WHITE)),
        float(board.has_kingside_castling_rights(chess.BLACK)),
        float(board.has_queenside_castling_rights(chess.BLACK)),
    ]
    return np.concatenate([flat_pieces, extras])  # 773

# =====================
# FUNCIÓN PARA GRAFO DE PARTIDA
# =====================

def pgn_to_graph(game):
    board = game.board()
    x = []
    edge_index = [[], []]
    node_idx = 0

    x.append(board_to_feature(board.copy()))

    for move in game.mainline_moves():
        board.push(move)
        x.append(board_to_feature(board.copy()))
        edge_index[0].append(node_idx)
        edge_index[1].append(node_idx + 1)
        node_idx += 1

    x = torch.tensor(np.stack(x), dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    return Data(x=x, edge_index=edge_index)

# =====================
# PIPELINE PRINCIPAL
# =====================

graphs = []

for zipname in tqdm(os.listdir(ZIP_FOLDER)):
    if not zipname.endswith(".zip"):
        continue

    player_name = zipname.replace(".zip", "")
    with zipfile.ZipFile(os.path.join(ZIP_FOLDER, zipname), 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_FOLDER)
    
    # Buscar archivo .pgn
    pgn_path = next(
        (os.path.join(EXTRACT_FOLDER, f) for f in os.listdir(EXTRACT_FOLDER) if f.endswith(".pgn")),
        None
    )
    if pgn_path is None:
        continue

    with open(pgn_path, encoding="utf-8", errors="ignore") as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break
            try:
                graph = pgn_to_graph(game)
                graph.player = player_name  # agregar nombre del jugador
                graphs.append(graph)
            except Exception as e:
                print(f"❌ Error procesando partida de {player_name}: {e}")
                continue

    # Elimina el .pgn temporal
    os.remove(pgn_path)

# =====================
# GUARDAR RESULTADOS
# =====================

torch.save(graphs, "chess_graphs.pt")
print(f"✅ {len(graphs)} grafos guardados en 'chess_graphs.pt'")

100%|██████████| 50/50 [18:35<00:00, 22.32s/it]


✅ 103553 grafos guardados en 'chess_graphs.pt'


In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
import pandas as pd
from tqdm import tqdm

# ======================
# Hiperparámetros
# ======================
INPUT_DIM = 773          # dimensión de features por nodo
HIDDEN_DIM = 256         # capa intermedia
EMBED_DIM = 128          # dimensión final del embedding
BATCH_SIZE = 32

# ======================
# Modelo GCN Encoder
# ======================
class GNNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return global_mean_pool(x, batch)

# ======================
# Cargar grafos
# ======================
graphs = torch.load("chess_graphs.pt")
loader = DataLoader(graphs, batch_size=BATCH_SIZE, shuffle=False)

model = GNNEncoder(INPUT_DIM, HIDDEN_DIM, EMBED_DIM)
model.eval()

# ======================
# Embedding loop
# ======================
embeddings = []
players = []

with torch.no_grad():
    for batch in tqdm(loader, desc="Generando embeddings"):
        z = model(batch.x, batch.edge_index, batch.batch)
        embeddings.append(z)
        players += batch.player

embeddings = torch.cat(embeddings).cpu().numpy()

# ======================
# Guardar resultados
# ======================
df = pd.DataFrame(embeddings, columns=[f"dim_{i}" for i in range(EMBED_DIM)])
df["player"] = players
df.to_csv("gnn_chess_embeddings.csv", index=False)
df.to_pickle("gnn_chess_embeddings.pkl")

print("✅ Embeddings GNN guardados como CSV y PKL.")

Generando embeddings: 100%|██████████| 3237/3237 [00:32<00:00, 98.28it/s] 


✅ Embeddings GNN guardados como CSV y PKL.
